In [ ]:
from astropy.io.fits import getdata, getheader
import astropy.units as u
from nustar_lunar_pointing.tracking import get_epoch_tle
from nustar_lunar_pointing.tracking import convert_nustar_time
from nustar_lunar_pointing.tracking import get_moon_j2000

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tlefile = '../data/NUSTAR_TLE_ARCHIVE.txt.2016122'
att = getdata('../data/nu30102028006A.attorb')
hdr = getheader('../data/nu30102028006A.attorb', 1)
mjdref = hdr['MJDREFI']

checktime = convert_nustar_time(att[0]['TIME'])
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)

In [ ]:
from astropy.coordinates import SkyCoord
doff = []
dra = []
ddec =[]
dt = []

base_ra = None
base_dec = None

tcheck = min(att['Time'])
step_size = 20. 
for ind, t0 in enumerate(att['Time']):
    if (t0 - tcheck) < step_size:
        continue
    else:
        tcheck = t0

    if (t0 - min(att['Time'])) > 3000:
        break

        
    t1 = convert_nustar_time(t0)
    
    ra_moon, dec_moon = get_moon_j2000(t1, line1, line2)
    moon_skycoord = SkyCoord(ra_moon, dec_moon)

    if base_ra is None:
        base_ra = ra_moon
        base_dec = dec_moon
        base_skycoord = moon_skycoord
        base_time = min(att['Time'])
    else:
        this_skycoord = SkyCoord(ra_moon, dec_moon)
        doff.extend([this_skycoord.separation(base_skycoord).arcsec])
        dra.extend([
            (ra_moon - base_ra).to(u.arcsec).value
        ])
        ddec.extend([
            (dec_moon - base_dec).to(u.arcsec).value
        ])
        dt.extend([t0 - base_time])
        


In [ ]:
plt.plot(dt, dra, 'g.')
plt.xlabel('Time (sec)')
plt.ylabel('Offset (arcsec)')

In [ ]:
from astropy.coordinates import SkyCoord
doff = []
dra = []
ddec =[]
dt = []

base_ra = None
base_dec = None

tcheck = min(att['Time'])
step_size = 1. # arcminute
last_skycoord = None
last_time = None

set = 0
for ind, t0 in enumerate(att['Time']):

    if (t0 - min(att['Time'])) < 1500:
        continue


    if (t0 - min(att['Time'])) > 1800:
        break

        
    t1 = convert_nustar_time(t0)
    
    ra_moon, dec_moon = get_moon_j2000(t1, line1, line2)
    moon_skycoord = SkyCoord(ra_moon, dec_moon)

    if set == 0:
        print(t1)
        
        if last_time is not None:
            print(t0 - last_time)
        
        last_time = t0
        last_skycoord = moon_skycoord


        set = 1
    else:
        if moon_skycoord.separation(last_skycoord).arcmin > step_size:
            set=0
               
    
    

In [ ]:
dra